In [1]:
import datetime
import locale

import datapane as dp
import pandas as pd

from zoneinfo import ZoneInfo

## Dates

We'll create `datetime_now` to simulate running `datetime.now()` at the start of the PyData slot.

In [2]:
datetime_now = datetime.datetime(
    2023, 4, 27, 10, 15, 0, 0, tzinfo=ZoneInfo("US/Pacific")
)
datetime_now

datetime.datetime(2023, 4, 27, 10, 15, tzinfo=zoneinfo.ZoneInfo(key='US/Pacific'))

## Currency

In [3]:
locale.setlocale(locale.LC_ALL, "en_US.UTF-8")

'en_US.UTF-8'

## Dataset

In [4]:
df_orders = pd.read_csv("data/order.csv", parse_dates=["Created at"])
df_items = pd.read_csv("data/items.csv", parse_dates=["Created at"], low_memory=False)
df_customers = pd.read_csv("data/cust.csv", parse_dates=["first_order", "last_order"])

## Window

Define a 30-day window.

In [5]:
def get_window(df, date_col, window_start, window_end):
    window = df[(df[date_col] > window_start) & (df[date_col] < window_end)]
    previous_period_window = df[
        (df[date_col] > window_start - (window_end - window_start))
        & (df[date_col] < window_start)
    ]
    return window, previous_period_window


window_end = datetime_now
window_start = datetime_now - datetime.timedelta(days=30)

df_orders_window, df_orders_window_previous = get_window(
    df_orders, "Created at", window_start, window_end
)
df_customers_window, df_customers_window_previous = get_window(
    df_customers, "first_order", window_start, window_end
)

print(
    f"{window_start.date()} to {window_end.date()} is {(window_end-window_start).days} days"
)

2023-03-28 to 2023-04-27 is 30 days


## Summary stats

30 day stats (sales, aov, new customers, new orders, etc.)

In [6]:
def summary_stats(df_orders, df_customers):
    stats = {}
    stats["orders"] = len(df_orders)
    stats["sales"] = len(df_orders[df_orders["Financial Status"] == "paid"])
    stats["aov"] = df_orders.Total.mean()
    stats["revenue"] = df_orders.Total.sum()
    stats["new_customers"] = len(df_customers)
    stats["returning_customers"] = (
        len(df_orders.Cust_ID.unique()) - stats["new_customers"]
    )
    return pd.DataFrame.from_dict(stats, orient="index").T


stats_current_period = summary_stats(df_orders_window, df_customers_window)
stats_previous_period = summary_stats(
    df_orders_window_previous, df_customers_window_previous
)
stats_delta = stats_current_period - stats_previous_period
stats_upward_change = stats_delta > 0

block_summary_stats = dp.Group(
    dp.BigNumber(
        "Orders Created",
        f"{stats_current_period['orders'].item():n}",
        f"{stats_delta['orders'].item():n}",
        is_upward_change=stats_upward_change["orders"].item(),
    ),
    dp.BigNumber(
        "Sales Completed",
        f"{stats_current_period['sales'].item():n}",
        f"{stats_delta['sales'].item():n}",
        is_upward_change=stats_upward_change["sales"].item(),
    ),
    dp.BigNumber(
        "New Customers",
        f"{stats_current_period['new_customers'].item():n}",
        f"{stats_delta['new_customers'].item():n}",
        is_upward_change=stats_upward_change["new_customers"].item(),
    ),
    dp.BigNumber(
        "Returning Customers",
        f"{stats_current_period['returning_customers'].item():n}",
        f"{stats_delta['returning_customers'].item():n}",
        is_upward_change=stats_upward_change["returning_customers"].item(),
    ),
    dp.BigNumber(
        "Revenue Generated",
        locale.currency(stats_current_period["revenue"].item(), grouping=True),
        locale.currency(stats_delta["revenue"].item(), grouping=True),
        is_upward_change=stats_upward_change["revenue"].item(),
    ),
    dp.BigNumber(
        "AOV",
        locale.currency(stats_current_period["aov"].item(), grouping=True),
        locale.currency(stats_delta["aov"].item(), grouping=True),
        is_upward_change=stats_upward_change["aov"].item(),
    ),
    columns=3,
)

block_summary_stats

## Audiences